In [2]:
import numpy as np
import os
import json
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

# Parámetros
IMG_WIDTH, IMG_HEIGHT = 28, 28
BATCH_SIZE = 64
NUM_CLASSES = 10  # Solo números del 0 al 9
EPOCHS = 30

# Función para cargar y procesar MNIST
def load_and_process_mnist():
    (x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
    # Expandir dimensiones para que sean compatibles con la CNN (altura, anchura, canales)
    x_train = np.expand_dims(x_train, -1).astype('float32') / 255.0
    x_test = np.expand_dims(x_test, -1).astype('float32') / 255.0
    # Invertir colores: números negros sobre fondo blanco
    x_train = 1.0 - x_train
    x_test = 1.0 - x_test
    return x_train, y_train, x_test, y_test

# Cargar datos
print("Cargando datos de MNIST...")
x_mnist_train, y_mnist_train, x_mnist_test, y_mnist_test = load_and_process_mnist()

# Convertir etiquetas a categóricas
y_mnist_train = to_categorical(y_mnist_train, num_classes=NUM_CLASSES)
y_mnist_test = to_categorical(y_mnist_test, num_classes=NUM_CLASSES)

# Dividir en entrenamiento y validación
x_train, x_val, y_train, y_val = train_test_split(
    x_mnist_train, y_mnist_train, test_size=0.2, random_state=42, stratify=y_mnist_train
)

print(f"Datos de entrenamiento: {x_train.shape[0]} muestras")
print(f"Datos de validación: {x_val.shape[0]} muestras")

# Generador para normalizar y aumentar las imágenes
train_datagen = ImageDataGenerator(
    rotation_range=10,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.1,
    horizontal_flip=False,
    fill_mode='nearest'
)

val_datagen = ImageDataGenerator()  # Solo rescale si es necesario, pero ya están normalizados

# Crear generadores
train_generator = train_datagen.flow(
    x_train,
    y_train,
    batch_size=BATCH_SIZE,
    shuffle=True
)

validation_generator = val_datagen.flow(
    x_val,
    y_val,
    batch_size=BATCH_SIZE,
    shuffle=False
)

# Construir la arquitectura de la CNN
model = Sequential()

# Primera capa convolucional
model.add(Conv2D(32, (3,3), activation='relu', input_shape=(IMG_WIDTH, IMG_HEIGHT, 1)))
model.add(BatchNormalization())
model.add(MaxPooling2D((2,2)))
model.add(Dropout(0.25))

# Segunda capa convolucional
model.add(Conv2D(64, (3,3), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D((2,2)))
model.add(Dropout(0.25))

# Tercera capa convolucional
model.add(Conv2D(128, (3,3), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D((2,2)))
model.add(Dropout(0.25))

# Aplanar
model.add(Flatten())

# Capa densa
model.add(Dense(512, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))

# Capa de salida
model.add(Dense(NUM_CLASSES, activation='softmax'))

# Compilar el modelo
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# Resumen del modelo
model.summary()

# Configurar callbacks
callbacks = [
    EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True),
    ModelCheckpoint('cnn_mnist_best.keras', monitor='val_loss', save_best_only=True)
]

# Entrenar el modelo
history = model.fit(
    train_generator,
    epochs=EPOCHS,
    validation_data=validation_generator,
    callbacks=callbacks
)

# Guardar el modelo entrenado
model.save('cnn_mnist_final.keras')

# Guardar los índices de clase en un archivo JSON
indice_a_clase = {i: str(i) for i in range(NUM_CLASSES)}

with open('cnn_mnist_class_indices.json', 'w') as f:
    json.dump(indice_a_clase, f)

print("Entrenamiento completado y modelo guardado exitosamente.")


Cargando datos de MNIST...
Datos de entrenamiento: 48000 muestras
Datos de validación: 12000 muestras


C:\Users\elias\anaconda3\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                      │ (None, 26, 26, 32)          │             320 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization                  │ (None, 26, 26, 32)          │             128 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d (MaxPooling2D)         │ (None, 13, 13, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 13, 13, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_1 (Conv2D)                    │ (None, 11, 11, 64)          │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_1                │ (None, 11, 11, 64)          │             256 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_1 (MaxPooling2D)       │ (None, 5, 5, 64)            │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 5, 5, 64)            │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_2 (Conv2D)                    │ (None, 3, 3, 128)           │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_2                │ (None, 3, 3, 128)           │             512 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_2 (MaxPooling2D)       │ (None, 1, 1, 128)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_2 (Dropout)                  │ (None, 1, 1, 128)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 512)                 │          66,048 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_3                │ (None, 512)                 │           2,048 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_3 (Dropout)                  │ (None, 512)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 10)                  │           5,130 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 166,794 (651.54 KB)

 Trainable params: 165,322 (645.79 KB)

 Non-trainable params: 1,472 (5.75 KB)

C:\Users\elias\anaconda3\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/30
750/750 ━━━━━━━━━━━━━━━━━━━━ 95s 116ms/step - accuracy: 0.6443 - loss: 1.2004 - val_accuracy: 0.9460 - val_loss: 0.1724
Epoch 2/30
750/750 ━━━━━━━━━━━━━━━━━━━━ 89s 119ms/step - accuracy: 0.8943 - loss: 0.3364 - val_accuracy: 0.9670 - val_loss: 0.1037
Epoch 3/30
750/750 ━━━━━━━━━━━━━━━━━━━━ 89s 118ms/step - accuracy: 0.9229 - loss: 0.2495 - val_accuracy: 0.9753 - val_loss: 0.0765
Epoch 4/30
750/750 ━━━━━━━━━━━━━━━━━━━━ 90s 120ms/step - accuracy: 0.9339 - loss: 0.2075 - val_accuracy: 0.9701 - val_loss: 0.0930
Epoch 5/30
750/750 ━━━━━━━━━━━━━━━━━━━━ 90s 120ms/step - accuracy: 0.9377 - loss: 0.1986 - val_accuracy: 0.9732 - val_loss: 0.0816
Epoch 6/30
750/750 ━━━━━━━━━━━━━━━━━━━━ 90s 120ms/step - accuracy: 0.9447 - loss: 0.1771 - val_accuracy: 0.9778 - val_loss: 0.0726
Epoch 7/30
750/750 ━━━━━━━━━━━━━━━━━━━━ 81s 108ms/step - accuracy: 0.9498 - loss: 0.1643 - val_accuracy: 0.9809 - val_loss: 0.0635
Epoch 8/30
750/750 ━━━━━━━━━━━━━━━━━━━━ 79s 106ms/step - accuracy: 0.9520 - loss: 0